# Import modules

In [2]:
import train
import environment as environment
#from neuralnet import convNet

Cuda: False


ImportError: cannot import name 'convNet'

# Set training and network parameters

In [ ]:
# training settings
trainingKwargs = {    
    'num_episodes' : 100,
    #rollout_limit = env.spec.timestep_limit # max rollout length
    'discount_factor' : 0.9, # reward discount factor (gamma), 1.0 = no discount
    'val_freq' : 25 # validation frequency
}

# training network settings
netKwargs = {
    'n_inputs' : 614,
    'n_hidden' : 500,
    #'n_outputs' : env.action_space.n, 
    'n_outputs' : 6, # This method is updated in the training class
    'learning_rate': 0.001,
    'batch_norm' : False,
    'conv1_in_channels' : 1,
    'conv1_out_channels' : 3,
    'conv2_out_channels' : 3,
    'conv3_out_channels' : 3,
    'kernel_size' : 5
}


# Instantiate environment, network, and trainer

In [ ]:
neuralNet = DQN(**netKwargs)
env = environment.randomEnv() # could be stopEnv, simpleEnv
valueTrainer = train.QTraining(env, neuralNet, **trainingKwargs)

# Start training

In [ ]:
valueTrainer.train()

# Visualize 

In [ ]:
print (env)